In [ ]:
# 원화마켓o, 유의종목x, BTCx 코인에 대해서
# 실시간으로 1분봉의 정보들을 가져오고
# 1초마다 체크해서
# 1분안에 이전 1분봉대비 10%가 오른 코인에 대해서
# 원화의 20%를 시장가매수하고
# 1초마다 체크해서
# 5%이상의 수익이 나오면 시장가매도를 한다.
# 계속 반복한다.

In [18]:
import requests, time, jwt, uuid, pyupbit, datetime
import pandas as pd
import numpy as np
from sendMail import send_email
import json

f = open("업비트정보.private.json", "r")
api_key = json.load(f)
f.close()

A_key = api_key["accessKey"]  # 본인 access_key 키로 변경
S_key = api_key["secretKey"]  # 본인 secret_key 키로 변경

In [10]:
tickers = pyupbit.get_tickers(fiat="KRW")
print(tickers)

['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-XRP', 'KRW-ETC', 'KRW-OMG', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-REP', 'KRW-ADA', 'KRW-SBD', 'KRW-POWR', 'KRW-BTG', 'KRW-ICX', 'KRW-EOS', 'KRW-TRX', 'KRW-SC', 'KRW-ONT', 'KRW-ZIL', 'KRW-POLYX', 'KRW-ZRX', 'KRW-LOOM', 'KRW-BCH', 'KRW-BAT', 'KRW-IOST', 'KRW-RFR', 'KRW-CVC', 'KRW-IQ', 'KRW-IOTA', 'KRW-HIFI', 'KRW-ONG', 'KRW-GAS', 'KRW-UPP', 'KRW-ELF', 'KRW-KNC', 'KRW-BSV', 'KRW-THETA', 'KRW-QKC', 'KRW-BTT', 'KRW-MOC', 'KRW-ENJ', 'KRW-TFUEL', 'KRW-MANA', 'KRW-ANKR', 'KRW-AERGO', 'KRW-ATOM', 'KRW-TT', 'KRW-CRE', 'KRW-MBL', 'KRW-WAXP', 'KRW-HBAR', 'KRW-MED', 'KRW-MLK', 'KRW-STPT', 'KRW-ORBS', 'KRW-VET', 'KRW-CHZ', 'KRW-STMX', 'KRW-DKA', 'KRW-HIVE', 'KRW-KAVA', 'KRW-AHT', 'KRW-LINK', 'KRW-XTZ', 'KRW-BORA', 'KRW-JST', 'KRW-CRO', 'KRW-TON', 'KRW-SXP', 'KRW-HUNT', 'KRW-PLA', 'KRW-DOT', 'KRW-SRM', 'KRW-MVL', 'KRW-STRAX', 'KRW-AQT', 'KRW-GLM', 'KRW-SSX'

In [28]:
for ticker in tickers:
    
    if ticker in ["KRW-BTC"]:
        continue
    
    print(ticker, pyupbit.get_current_price(ticker))
    
    ohlcv_value = pyupbit.get_ohlcv(ticker, interval="minute1")
    ## 앞 뒤 "종가" 차이를 변화량 "change"로 지정
    ohlcv_value['change'] = ohlcv_value['close'] - ohlcv_value['close'].shift(1)
    ohlcv_value['change_rate'] = (ohlcv_value['close'] / ohlcv_value['close'].shift(1))
    
    print(ohlcv_value)
    time.sleep(1)

KRW-ETH 2081000
                          open       high        low      close     volume  \
2023-02-05 19:24:00  2100000.0  2100000.0  2100000.0  2100000.0   6.279768   
2023-02-05 19:25:00  2100000.0  2100000.0  2098000.0  2098000.0  15.655573   
2023-02-05 19:26:00  2098000.0  2100000.0  2098000.0  2100000.0   9.028033   
2023-02-05 19:27:00  2100000.0  2101000.0  2097000.0  2098000.0   7.336186   
2023-02-05 19:28:00  2098000.0  2098000.0  2096000.0  2097000.0   9.579749   
...                        ...        ...        ...        ...        ...   
2023-02-05 22:39:00  2082000.0  2082000.0  2081000.0  2082000.0  39.917632   
2023-02-05 22:40:00  2082000.0  2082000.0  2080000.0  2081000.0  41.260032   
2023-02-05 22:41:00  2081000.0  2081000.0  2080000.0  2081000.0  16.491868   
2023-02-05 22:42:00  2080000.0  2082000.0  2079000.0  2082000.0  19.608765   
2023-02-05 22:43:00  2080000.0  2081000.0  2080000.0  2081000.0   4.345640   

                            value  change  chan

KeyboardInterrupt: 

In [21]:
## API로 업비트에서 내 계좌 조회
my_exchange_account = pd.DataFrame(requests.get("https://api.upbit.com/v1/accounts",
                                                    headers={"Authorization": 'Bearer {}'.format(
                                                        jwt.encode({'access_key': A_key,
                                                                    'nonce': str(uuid.uuid4())}, S_key))}).json())
now_krw = float(my_exchange_account[my_exchange_account['currency'] == 'KRW']['balance'].tail())

In [22]:
now_krw

57957.47121724

In [23]:
df = pyupbit.get_ohlcv(ticker, interval="minute1", count=1)

# Print the result
print(df.iloc[0]['volume'])

34543.45162571


In [24]:
ohlcv_value = pyupbit.get_ohlcv(market_code, interval="minute1")
## 앞 뒤 "종가" 차이를 변화량 "change"로 지정
ohlcv_value['change'] = ohlcv_value['close'] - ohlcv_value['close'].shift(1)
print(ohlcv_value)

NameError: name 'market_code' is not defined